In [2]:
from datetime import datetime

#Person Class
class Person:
    
    #Only Default value needed is name, Friends and Posts are lists to more easily sort sub data
    # it is also for it to be easier to combine and sort when combining multiple users posts
    #Would usually use postgres to store this data and do requests for the data
    def __init__(self,name=None):
        self.name=name
        self.friends=[]
        self.posts=[]
        
    #would make statuses for friends to accept or deny request
    def add_friend(self,friend):
        if friend not in self.friends:
            self.friends.append(friend)
            friend.friends.append(self)
            return friend.get_name()+" Added as Friend"
        return True
        
    def remove_friend(self,friend):
        if friend in self.friends:
            self.friends.remove(friend)
            friend.friends.remove(self)
            return friend.get_name()+" Removed as Friend"
        return True
    
    
    
    def add_post(self,post):
        self.posts.append([post,datetime.now()])
    
    def remove_post(self,post):
        self.posts=[x for x in self.posts if x[0]!= post]

        
    def get_name(self):
        return self.name
    
    def get_friends(self):
        return self.friends
    
    def get_posts(self):
        return self.posts
    
    
    
    
    
    

In [3]:
from datetime import datetime,timedelta

#Converts times to show correct message
def time_difference(test_date):
    current_time=datetime.now()
    total_seconds=current_time-test_date
    total_seconds=total_seconds.total_seconds()
    if total_seconds>=86400:
        return "{0} Days Ago".format(int(round(total_seconds/86400,0)))
    if total_seconds<60:
        return "{0} Seconds Ago".format(int(round(total_seconds,0)))
    if total_seconds<3600:
        return "{0} Minutes Ago".format(int(round(total_seconds/60,0)))
    if total_seconds<86400:
        return "{0} Hours Ago".format(int(round(total_seconds/3600,0)))
    
    


    
#get post of all users listed, if not a list turns it into one    
def get_posts_of_users(user_list):
    if type(user_list) is not list:
        user_list=[user_list]
    posts=[]
    #go through all users listed and combines posts
    for user in user_list:
        user_posts=user.get_posts()
        user_name=user.get_name()
        for user_post in user_posts:
            #Username, User, Post, Post Time, Time Difference message
            posts.append([user_name,user,user_post[0],user_post[1],time_difference(user_post[1])])
    #sorts it based on post time, and get newest first
    posts.sort(key=lambda x:x[3],reverse=True)
    
    return posts


#Get all posts of 1 user
def get_user_posts(User):
    return get_posts_of_users(User)


#Get all posts from a users friends
def get_friends_posts(User):
    return get_posts_of_users(User.get_friends())


#Get all posts including the users
def get_all_posts(User):
    users=User.get_friends()
    users.append(User)
    return get_posts_of_users(users)


        
        
        
    
    
    
    

In [11]:
import unittest
import time
#Unit Tests
class Test_Post_and_Friend(unittest.TestCase):
    
    #SetUp For Tests to Reuse
    def setUp(self):
        self.amy=Person("AMY")
        self.billy=Person("BILLY")
        self.tom=Person("TOM")
        self.troy=Person('TROY')
        self.billy.add_post("Test Post 1")
        time.sleep(.1)
        self.amy.add_post("Test Post 2")
        time.sleep(.1)
        self.tom.add_post("Test Post 3")
        time.sleep(.1)
        self.billy.add_post("Test Post 4")
        self.troy=Person("Troy")
        self.amy.add_friend(self.billy)
        
    #Test Time Conversion Works Correctly
    def test_time_conversion(self):
        self.assertEqual(time_difference(datetime.now()-timedelta(days=1)),"1 Days Ago")
        self.assertEqual(time_difference(datetime.now()-timedelta(seconds=5)),"5 Seconds Ago")
        self.assertEqual(time_difference(datetime.now()-timedelta(minutes=2)),"2 Minutes Ago")
        self.assertEqual(time_difference(datetime.now()-timedelta(hours=1)),"1 Hours Ago")
        
        
    
    def test_remove_friend(self):
        self.assertEqual(self.tom.remove_friend(self.billy),True)
        self.assertEqual(self.amy.remove_friend(self.billy),"BILLY Removed as Friend") 
        
        
    def test_friend_add(self):
        self.assertEqual(self.amy.add_friend(self.billy),True)
        self.assertEqual(self.tom.add_friend(self.billy),'BILLY Added as Friend')
        
        
    
    
    def test_add_post(self):
        test_post="Testing Post"
        self.troy.add_post(test_post)
        self.assertEqual(self.troy.get_posts()[0][0],test_post)
    
    def test_remove_post(self):
        test_post="Testing Post"
        self.troy.add_post(test_post)
        self.assertEqual(self.troy.get_posts()[0][0],test_post)
        self.troy.remove_post(test_post)
        self.assertEqual(self.troy.get_posts(),[])
        
    def test_get_user_posts(self):
        billys_posts=get_posts_of_users(self.billy)
        self.assertEqual(billys_posts[0][2],'Test Post 4')
        self.assertEqual(billys_posts[1][2],'Test Post 1')
        self.assertEqual(get_posts_of_users(self.troy),[])
        
    def test_get_friends_posts(self):
        
        amys_friends_posts=get_friends_posts(self.amy)
        self.assertEqual(amys_friends_posts[0][2],'Test Post 4')
        self.assertEqual(amys_friends_posts[1][2],'Test Post 1')
        self.assertEqual(get_friends_posts(self.tom),[])

    def test_get_all_posts(self):
        
        billys_posts=get_all_posts(self.billy)
        self.assertEqual(billys_posts[0][2],'Test Post 4')
        self.assertEqual(billys_posts[1][2],'Test Post 2')
        self.assertEqual(billys_posts[2][2],'Test Post 1')
        self.assertEqual(get_all_posts(self.tom)[0][2],"Test Post 3")
        
        self.assertEqual(get_all_posts(self.troy),[])
        
    
        
        



In [12]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

........
----------------------------------------------------------------------
Ran 8 tests in 2.604s

OK
